<a href="https://colab.research.google.com/github/phys-ken/ypc_database/blob/master/ypc_database_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YPCの例会アルバムをスクレイピングする
* [YPC過去の例会アルバム](https://www2.hamajima.co.jp/~tenjin/ypc/ypcalbum.html)にアクセスして、全部のリンク先の記事にアクセスして、文書データのみをcsvで出力します。大体、20分くらいかかります。

### 技術的な解説
* pythonのBeautifulsoup4でスクレイピングし、改行コード等はテキスト処理で切り取っています。
* pandasにデータフレームとして呼び込み、行列を整理し出力します。
* [こちら](https://phys-ken.github.io/ypc_database/README.html)のページで、csvを検索することが可能になっています。

In [1]:
#!pip install chardet
#文字化け対策(https://orangain.hatenablog.com/entry/encoding-in-requests-and-beautiful-soup)
#!pip install tqdm

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import tqdm

In [3]:
url = "https://www2.hamajima.co.jp/~tenjin/ypc/ypcalbum.html"
r = requests.get(url)
time.sleep(3)

In [4]:
soup = BeautifulSoup(r.content, "html.parser")

In [5]:
titles_html = soup.find_all("dt")
titles = []
for title in titles_html:
    titles.append(title.find("a"))

In [6]:
top_url = "https://www2.hamajima.co.jp/~tenjin/ypc/"

url_list = []
for title in titles:
    url_list.append([title.get_text(),top_url + title.get("href")])

In [7]:
print("全部で" + str(len(url_list)) + "回処理を行います、")

for i in tqdm.tqdm(range(len(url_list))):
    r = requests.get(url_list[i][1])
    time.sleep(3)
    soup =  BeautifulSoup(r.content, "html.parser")

    ps =soup.find_all("p")

    strip_p = []
    for p in ps:
        if p.get("align") == None:
            strip_p.append(p)

    theme_list = []
    counter = -1


    #文字列を処理する。改行タグやスペース、\rタグをはずす
    for p in strip_p:
        try:
            if p.find("font").get("size") == ("+3"):
                counter += 1
                theme_list.append([p.find("font").get_text()])
                theme_list[counter].append("")
                theme_list[counter][1] += p.get_text().strip().replace( '\n' , '.' ).replace( '\r' , '.' )
            else:
                theme_list[counter][1] += p.get_text().strip().replace( '\n' , '.' ).replace( '\r' , '.' )
        except:
            try:
                theme_list[counter][1] += p.get_text().strip().replace( '\n' , '.' ).replace( '\r' , '.' )
            except:
                pass

    url_list[i].append(theme_list)
    #print("\r" + str(i).zfill(5)+"回目の処理を終えました！" , end="")

print("スクレイピングが終了しました。")

全部で326回処理を行います、


100%|██████████| 326/326 [19:36<00:00,  3.61s/it]

スクレイピングが終了しました。


In [8]:
#空のデータフレームをはじめに作って、そこにどんどん結合していく
import pandas as pd
df = pd.DataFrame()

for i in range(len(url_list)):
    df_tmp = pd.DataFrame(url_list[i][2],columns=["発表テーマ","本文"])
    df_tmp["日時"] = url_list[i][0]
    df_tmp["URL"] = url_list[i][1]
    df = pd.concat([df,df_tmp])

df = df.reset_index()
df = df.drop(df.columns[[0]], axis=1)
df

,発表テーマ,本文,日時,URL
0,授業研究：MetaMojiを使う理科授業,授業研究：MetaMojiを使う理科授業 益田さんの発表 . MetaMojiは授業支援シス...,ＹＰＣ2024年10月例会(10/14 関東学院中・高等学校・Zoomハイブリッド）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc24x...
1,酸化と還元,酸化と還元 天野さんの発表. インジゴカルミンは「青色２号」として食品添加物にも使われる色素...,ＹＰＣ2024年10月例会(10/14 関東学院中・高等学校・Zoomハイブリッド）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc24x...
2,正しいモーター,正しいモーター 曽谷さんの発表. 前回例会で発表の「気まぐれモーター」は整流子がないのに気ま...,ＹＰＣ2024年10月例会(10/14 関東学院中・高等学校・Zoomハイブリッド）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc24x...
3,AMラジオ少年団バージョン,AMラジオ少年団バージョン 山本の発表. 今年の８月例会でおもしろ科学たんけん工房の松本聡さ...,ＹＰＣ2024年10月例会(10/14 関東学院中・高等学校・Zoomハイブリッド）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc24x...
4,LANケーブル工作,LANケーブル工作 益田さんの発表. 以前、千葉高校の阿部さんが2024年2月例会で紹介して...,ＹＰＣ2024年10月例会(10/14 関東学院中・高等学校・Zoomハイブリッド）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc24x...
...,...,...,...,...
4207,飛び出す色環,飛び出す色環 喜多さんの発表例の立体メガネChromaDepth３Ｄで見ると中央が飛び出して...,ＹＰＣ1997年１１月例会（11/12柿生西高校）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc97y...
4208,レーザー光線で音の振動波形を見る,レーザー光線で音の振動波形を見る右近さんが披露してくれた実験。ミラーキューブを空冷ファンに接...,ＹＰＣ1997年10月例会（10/22慶應義塾高校）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc97x...
4209,ビー玉による圧力のモデル,ビー玉による圧力のモデル 水上さんの科学の祭典ステージ紹介上のレールから台はかりの上にビー...,ＹＰＣ1997年８月例会（8/30県立厚木高校）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc978...
4210,手作りラブメータ（ハンドボイラ）,手作りラブメータ（ハンドボイラ） 高杉さん紹介資料瓶とエーテルで自作できます。..こちらは厚...,ＹＰＣ1997年８月例会（8/30県立厚木高校）,https://www2.hamajima.co.jp/~tenjin/ypc/ypc978...


In [9]:
#タイムスタンプをつけてcsv出力＆ダウンロード
import datetime
from google.colab import files
now = datetime.datetime.now()
filename = 'ypclog_' + now.strftime('%Y%m%d_%H%M%S') + '.csv'
print(filename)
df.to_csv(filename , encoding = 'utf-8-sig')
files.download(filename)

ypclog_20241216_231114.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>